**Group 5: Jacob Berkel, Lee Kim, Andy Nguyen**<br>
**Topic: Protein Structure Sequence**

In [1]:
!pip install pyGMS

In [2]:
import pyGMs as gm
import numpy as np
import matplotlib.pyplot as plt
import os.path
from sklearn.model_selection import train_test_split

%matplotlib inline

## Preprocessing, loading, and splitting the data

In [3]:
# Preprocessing the data
if not os.path.exists('X.txt'):
    with open('ss.txt','r') as f: text  = f.read();
    no_nl = text.replace('\n',"");
    splt = no_nl.split('>')
    splt = splt[1:]  # remove initial blank
    m = int(len(splt)/2)

    T,X,Y = [None]*m,[None]*m,[None]*m
    for i in range(m):
        T[i] = splt[2*i][:4]
        X[i] = splt[2*i][15:]
        Y[i] = splt[2*i+1][13:]

    n = len(X)
    nkeep = 5000
    idx = np.random.permutation(n)[:nkeep]

    with open('X.txt',"w") as f: f.writelines(X[i]+"\n" for i in idx);
    with open('Y.txt',"w") as f: f.writelines(Y[i]+"\n" for i in idx);
    with open('T.txt',"w") as f: f.writelines(T[i]+"\n" for i in idx);

# Loading the data
with open('X.txt') as f: Xs = f.read().split('\n');
with open('Y.txt') as f: Ys = f.read().split('\n');
with open('T.txt') as f: Ts = f.read().split('\n');
X = [x for x in Xs if len(x)]
Y = [y for y in Ys if len(y)]
T = [t for t in Ts if len(t)]

# Amino acid
i2aa = 'ACDEFGHIKLMNPQRSTUVWXY'
aa2i = {c:i for i,c in enumerate(i2aa)}
dx = len(i2aa)

# Structure only: p(yt|yt-1) * p(xt|yt)
i2ss = ' BEGHIST'
ss2i = {c:i for i,c in enumerate(i2ss)}
dy = len(i2ss)
small = {' ': 0, 'B': 1, 'E':1, 'G':2,'H':2,'I':2, 'S':3,'T':2}

# Structure and protein:  p(yt,xt|yt-1,xt-1)
i2ss2 = [s+a for s in i2ss for a in i2aa]
ss22i = {c:i for i,c in enumerate(i2ss2)}
ss22e = {i:small[c[0]] for i,c in enumerate(i2ss2)}
dy2 = len(i2ss2)

Y2= [ np.array([ss22i[c+a] for c,a in zip(y,x)],dtype=int) for y,x in zip(Y,X)]
X = [ np.array([aa2i[c] for c in x],dtype=int) for x in X]
Y = [ np.array([ss2i[c] for c in y],dtype=int) for y in Y]

# Splitting the data into training and testing sets
# Set 1: 80% training, 20% testing
XTr1, XTe1, YTr1, YTe1 = train_test_split(X, Y, test_size=0.2, random_state=42)
# Set 2: 20% training, 80% testing
XTr2, XTe2, YTr2, YTe2 = train_test_split(X, Y, test_size=0.8, random_state=42)

## Functions for ML estimation, score, and prediction

In [4]:
# Estimate output (emission) probabilities given latent state Z
def est_O(X,Z, dx,dz):
    pZX = np.zeros((dz,dx)) + 1e-20
    for x,z in zip(X,Z):
        for t in range(len(z)):
            pZX[z[t],x[t]] += 1.
    pZX /= pZX.sum(axis=1,keepdims=True)
    return pZX

# Estimate output for HMM using pairs
def est_O_pairs(X, Z, dx, dz):
    pZX = np.zeros((dz, dx, dx)) + 1e-20
    for x, z in zip(X, Z):
        for t in range(1, len(z)):
            pZX[z[t], x[t], x[t-1]] += 1.
    pZX /= pZX.sum(axis=1, keepdims=True)
    return pZX

# Estimate transition probabilities given latent states Z
def est_T(X,Z, dx,dz):
    pZZ = np.zeros((dz,dz)) + 1e-4
    for z in Z:
        for t in range(len(z)-1):
            pZZ[z[t],z[t+1]] += 1
    pZZ /= pZZ.sum(axis=1,keepdims=True)
    return pZZ

# Estimate initial state distribution from sequences Z
def est_P0(X,Z, dx,dz):
    p0 = np.zeros(dz)
    for z in Z: p0[z[0]] += 1;
    return p0/p0.sum()

# Evaluate the log-likelihood of (complete) data
def eval_LL(p0,T,O, X,Z):
    lnT, lnO = np.log(T), np.log(O)
    L = np.zeros( (len(X),) );
    for i in range(len(X)):
        x,z = X[i],Z[i]
        lnP = np.log(p0[z[0]]) + lnO[z[0],x[0]]
        for t in range(1,len(x)): lnP += lnT[z[t-1],z[t]] + lnO[z[t],x[t]]
        L[i] = lnP;
    return L

# Evaluate the log-likelihood for HMM using pairs
def eval_LL_pairs(p0, T, O, X, Z):
    lnT, lnO = np.log(T), np.log(O)
    L = np.zeros(len(X))
    for i in range(len(X)):
        x, z = X[i], Z[i]
        lnP = np.log(p0[z[0]]) + lnO[z[0], x[0], x[0]]
        for t in range(1, len(x)):
            lnP += lnT[z[t-1], z[t]] + lnO[z[t], x[t], x[t-1]]
        L[i] = lnP
    return L

# Predict the most likely latent state sequence
def predict(p0,T,O, X,Z):
    Zhat = [None]*len(X); dz,dx = O.shape;
    for i,x in enumerate(X):
        n = len(x);
        R = np.zeros((dz,n))  # store backward messages
        R[:,n-1] = O[:,x[n-1]]
        for t in range(n-2,-1,-1):
            R[:,t]  = (T*R[:,t+1]).max(1) * O[:,x[t]]
            R[:,t] /= R[:,t].max();
        zhat = np.zeros((n,),dtype=int);
        zhat[0] = (p0*R[:,0]).argmax();
        for t in range(1,n):
            zhat[t] = (T[zhat[t-1],:]*R[:,t]).argmax();
        Zhat[i] = zhat; #np.array([zhat[i] for i in range(n)],dtype=int)
    return Zhat

# Predict the most likely latent state sequence for HMM using pairs
def predict_pairs(p0, T, O, X, Z):
    Zhat = [None] * len(X)
    dz, dx, _ = O.shape
    for i, x in enumerate(X):
        n = len(x)
        R = np.zeros((dz, n))
        R[:, n-1] = O[:, x[n-1], x[n-1]]
        for t in range(n-2, -1, -1):
            R[:, t] = (T @ R[:, t+1]) * O[:, x[t], x[t-1]]
            R[:, t] /= R[:, t].max()
        zhat = np.zeros(n, dtype=int)
        zhat[0] = (p0 * R[:, 0]).argmax()
        for t in range(1, n):
            zhat[t] = (T[zhat[t-1], :] * R[:, t]).argmax()
        Zhat[i] = zhat
    return Zhat

# Evaluate the average error rate between two collections of sequences
def err(Y,Yhat):
    er,nr = 0.,0.
    for y,yh in zip(Y,Yhat):
        er += np.sum(np.array([ss22e[yi] for yi in y]) != np.array([ss22e[yi] for yi in yh]))
        nr += len(y)
    er /= nr
    return er;

# Error function for the n-gram models
def error(Y,Yhat):
    errs = []
    diffs = np.count_nonzero(Y - Yhat)
    er = 1-(np.sum(diffs)/len(Y))
    errs.append(er)
    return np.average(errs)

# Comparing Performance of N-Gram Models

## 1. Hidden Markov Model


In [5]:
%%time
# Simple HMM where the latent variable is the secondary structure label
O = est_O(X,Y, dx,dy);
T = est_T(X,Y, dx,dy);
p0= est_P0(X,Y,dx,dy)
print("Simple HMM")
print("\nLog-likelihood:", np.mean(eval_LL(p0,T,O,X,Y)))
print("Error rate:", err(Y, predict(p0,T,O,X,Y)))
print()

Simple HMM

Log-likelihood: -982.9408452682007
Error rate: 0.0

CPU times: user 19.5 s, sys: 157 ms, total: 19.7 s
Wall time: 20.5 s


In [6]:
%%time
# HMM using pairs of amino acids and both structure and protein sequence
O2 = est_O_pairs(X,Y2,dx,dy2);
T2 = est_T(X,Y2,dx,dy2);
p02= est_P0(X,Y2,dx,dy2)
p02 = np.array(p02.dot(np.matrix(T2)**50))[0]
print("Pair HMM")
print("\nLog-likelihood:", np.mean(eval_LL_pairs(p02,T2,O2,X,Y2)))
print("Error rate:", err(Y2, predict_pairs(p02,T2,O2,X,Y2)))
print()

Pair HMM

Log-likelihood: -956.9609834137518
Error rate: 0.5533346543406522

CPU times: user 2min 6s, sys: 57.6 s, total: 3min 3s
Wall time: 38.4 s


## 2. Amino Acid Bigram Model
Based on the current amino acid, this predicts the most likely secondary structure.

In [7]:
aa = list(aa2i.keys())
ss = list(ss2i.keys())
aavals = list(aa2i.values())
ssvals = list(ss2i.values())

#### Train/Test Set 1 (80:20)

In [8]:
# Count frequency of each transition for all proteins in training data
T_aa = np.zeros( (len(aavals), len(ssvals)) )
for j in range(len(XTr1)):
    for i in range(len(XTr1[j])):
        T_aa[ aavals.index(XTr1[j][i]), ssvals.index(YTr1[j][i]) ] += 1
T_aa/= T_aa.sum(axis=1, keepdims=True) # normalize to get P(Yt |Xt)
assert((T_aa>=0).all())

In [9]:
# Given an amino acid sequence aaseq,
# determine the secondary structure sequence ssseq
def aa_bigram_sample(aaseq):
    ssseq = np.array([])
    for i in range(len(aaseq)):
        ssseq = np.append(ssseq, np.random.choice(ssvals, p=T_aa[aaseq[i],:]) )
    return ssseq

In [10]:
%%time
errors = []
trials = 10
for i in range(len(XTe1)):
    ers = []
    for j in range(trials):
        ers.append(error(aa_bigram_sample(XTe1[i]),YTe1[i]))
    errors.append(np.average(ers))
print("Amino Acid Bigram Model (80:20)")
print("\nError rate:", np.average(errors))
print()

Amino Acid Bigram Model (80:20)

Error rate: 0.24730012724612446

CPU times: user 1min 26s, sys: 626 ms, total: 1min 27s
Wall time: 1min 30s


#### Train/Test Set 2 (20:80)

In [11]:
# Count frequency of each transition for all proteins in training data
T_aa = np.zeros( (len(aavals), len(ssvals)) )
for j in range(len(XTr2)):
    for i in range(len(XTr2[j])):
        T_aa[ aavals.index(XTr2[j][i]), ssvals.index(YTr2[j][i]) ] += 1
T_aa/= T_aa.sum(axis=1, keepdims=True) # normalize to get P(Yt |Xt)
assert((T_aa>=0).all())

In [12]:
# Given an amino acid sequence aaseq,
# determine the secondary structure sequence ssseq
def aa_bigram_sample(aaseq):
    ssseq = np.array([])
    for i in range(len(aaseq)):
        ssseq = np.append(ssseq, np.random.choice(ssvals, p=T_aa[aaseq[i],:]) )
    return ssseq

In [13]:
%%time
errors = []
trials = 10
for i in range(len(XTe2)):
    ers = []
    for j in range(trials):
        ers.append(error(aa_bigram_sample(XTe2[i]),YTe2[i]))
    errors.append(np.average(ers))
print("Amino Acid Bigram Model (20:80)")
print("\nError rate:", np.average(errors))
print()

Amino Acid Bigram Model (20:80)

Error rate: 0.24686848898502967

CPU times: user 5min 22s, sys: 2.03 s, total: 5min 24s
Wall time: 5min 33s


# 3. Secondary Structure Bigram Model
This model predicts the first secondary structure based on the first amino acid, then predicts the rest of the secondary structure based on what secondary structure is most likely to follow the previous. This model is very unlikely to perform well.

#### Train/Test Set 1 (80:20)

In [14]:
T_ss = np.zeros( (len(ssvals), len(ssvals)))
for j in range(len(YTr1)):
    for i in range(len(YTr1[j])):
        T_ss[ ssvals.index(YTr1[j][i-1]), ssvals.index(YTr1[j][i]) ] += 1
T_ss/= T_ss.sum(axis=1, keepdims=True) # normalize to get P(Yt |Xt)
assert((T_ss>=0).all())

In [15]:
# Given an amino acid sequence aaseq,
# determine the secondary structure sequence ssseq
def ss_bigram_sample(aaseq):
    ssseq = np.array([np.random.choice(ssvals, p=T_aa[aaseq[0],:])])
    for i in range(1,len(aaseq)):
        ssseq = np.append(ssseq, np.random.choice(ssvals, p=T_ss[ssseq[i-1],:]) )
    return ssseq

In [16]:
%%time
errors = []
trials = 10
for i in range(len(XTe1)):
    ers = []
    for j in range(trials):
        ers.append(error(ss_bigram_sample(XTe1[i]),YTe1[i]))
    errors.append(np.average(ers))
print("Secondary Structure Bigram Model (80:20)")
print("\nError rate:", np.average(errors))
print()

Secondary Structure Bigram Model (80:20)

Error rate: 0.22385058188411877

CPU times: user 1min 20s, sys: 385 ms, total: 1min 20s
Wall time: 1min 21s


#### Train/Test Set 2 (20:80)

In [17]:
T_ss = np.zeros( (len(ssvals), len(ssvals)))
for j in range(len(YTr2)):
    for i in range(len(YTr2[j])):
        T_ss[ ssvals.index(YTr2[j][i-1]), ssvals.index(YTr2[j][i]) ] += 1
T_ss/= T_ss.sum(axis=1, keepdims=True) # normalize to get P(Yt |Xt)
assert((T_ss>=0).all())

In [18]:
# Given an amino acid sequence aaseq,
# determine the secondary structure sequence ssseq
def ss_bigram_sample(aaseq):
    ssseq = np.array([np.random.choice(ssvals, p=T_aa[aaseq[0],:])])
    for i in range(1,len(aaseq)):
        ssseq = np.append(ssseq, np.random.choice(ssvals, p=T_ss[ssseq[i-1],:]) )
    return ssseq

In [19]:
%%time
errors = []
trials = 10
for i in range(len(XTe2)):
    ers = []
    for j in range(trials):
        ers.append(error(ss_bigram_sample(XTe2[i]),YTe2[i]))
    errors.append(np.average(ers))
print("Secondary Structure Bigram Model (20:80)")
print("\nError rate:", np.average(errors))
print()

Secondary Structure Bigram Model (20:80)

Error rate: 0.22246514223958092

CPU times: user 5min 10s, sys: 1.6 s, total: 5min 11s
Wall time: 5min 14s


# 4. Amino Acid Trigram Model
This model takes into consideration the previous two amino acids and uses them to predict the next most likely secondary structure.

#### Train/Test Set 1 (80:20)

In [20]:
# Count frequency of each transition for all proteins in training data
T2 = np.zeros( (len(aavals), len(aavals), len(ssvals)) )
for j in range(len(XTr1)):
    for i in range(0, len(XTr1[j])):
        if i == 0:
            T2[ 0 , aavals.index(XTr1[j][i]), ssvals.index(YTr1[j][i]) ] += 1
        else:
            T2[ aavals.index(XTr1[j][i-1]), aavals.index(XTr1[j][i]), ssvals.index(YTr1[j][i]) ] += 1
T2 /= T2.sum(axis=2, keepdims=True) # normalize
np.nan_to_num(T2,copy=False)
# Removing and correcting nan entries
a,b,c = T2.shape
avg = 1 / (c)
for i in range(a):
    for j in range(b):
        if abs(1 - sum(T2[i][j])) > 0.0001:
            T2[i][j] = np.full((c), avg)
assert((T2>=0).all())

In [21]:
# Given an amino acid sequence aaseq,
# determine the secondary structure sequence ssseq
def aa_trigram_sample(aaseq):
    ssseq = np.array([])
    for i in range(0,len(aaseq)):
        if i == 0:
            ssseq = np.append(ssseq, np.random.choice(ssvals, p=T_aa[aaseq[i],:]) )
        else:
            ssseq = np.append(ssseq, np.random.choice(ssvals, p=T2[aaseq[i-1],aaseq[i],:]) )
    return ssseq

In [22]:
%%time
# Trained on random selection of 80% of the data set
errors = []
trials = 10
for i in range(len(XTe1)):
    ers = []
    for j in range(trials):
        ers.append(error(aa_trigram_sample(XTe1[i]),YTe1[i]))
    errors.append(np.average(ers))
print("Amino Acid Trigram Model (80:20)")
print("\nError rate:", np.average(errors))
print()

Amino Acid Trigram Model (80:20)

Error rate: 0.2665509523622427

CPU times: user 1min 25s, sys: 433 ms, total: 1min 26s
Wall time: 1min 27s


#### Train/Test Set 2 (20:80)

In [23]:
# Count frequency of each transition for all proteins in training data
T2 = np.zeros( (len(aavals), len(aavals), len(ssvals)) )
for j in range(len(XTr2)):
    for i in range(0, len(XTr2[j])):
        if i == 0:
            T2[ 0 , aavals.index(XTr2[j][i]), ssvals.index(YTr2[j][i]) ] += 1
        else:
            T2[ aavals.index(XTr2[j][i-1]), aavals.index(XTr2[j][i]), ssvals.index(YTr2[j][i]) ] += 1
T2 /= T2.sum(axis=2, keepdims=True) # normalize
np.nan_to_num(T2,copy=False)
# Removing and correcting nan entries
a,b,c = T2.shape
avg = 1 / (c)
for i in range(a):
    for j in range(b):
        if abs(1 - sum(T2[i][j])) > 0.0001:
            T2[i][j] = np.full((c), avg)
assert((T2>=0).all())

In [24]:
# Given an amino acid sequence aaseq,
# determine the secondary structure sequence ssseq
def aa_trigram_sample(aaseq):
    ssseq = np.array([])
    for i in range(0,len(aaseq)):
        if i == 0:
            ssseq = np.append(ssseq, np.random.choice(ssvals, p=T_aa[aaseq[i],:]) )
        else:
            ssseq = np.append(ssseq, np.random.choice(ssvals, p=T2[aaseq[i-1],aaseq[i],:]) )
    return ssseq

In [25]:
%%time
# Trained on random selection of 80% of the data set
errors = []
trials = 10
for i in range(len(XTe2)):
    ers = []
    for j in range(trials):
        ers.append(error(aa_trigram_sample(XTe2[i]),YTe2[i]))
    errors.append(np.average(ers))
print("Amino Acid Trigram Model (20:80)")
print("\nError rate:", np.average(errors))
print()

Amino Acid Trigram Model (20:80)

Error rate: 0.265099561111494

CPU times: user 5min 14s, sys: 1.52 s, total: 5min 16s
Wall time: 5min 19s


# 5. Amino Acid Quadgram Model
This model takes into consideration the previous three amino acids and uses them to predict the next most likely secondary structure.

#### Train/Test Set 1 (80:20)

In [26]:
# Count frequency of each transition for all proteins in training data
T3 = np.zeros( (len(aavals), len(aavals), len(aavals), len(ssvals)) )
for j in range(len(XTr1)):
    for i in range(0, len(XTr1[j])):
        if i == 0:
            T3[ 0 , 0, aavals.index(XTr1[j][i]), ssvals.index(YTr1[j][i]) ] += 1
        elif i==1:
            T3[ 0, aavals.index(XTr1[j][i-1]), aavals.index(XTr1[j][i]), ssvals.index(YTr1[j][i]) ] += 1
        else:
            T3[ aavals.index(XTr1[j][i-2]), aavals.index(XTr1[j][i-1]), aavals.index(XTr1[j][i]), ssvals.index(YTr1[j][i]) ] += 1
T3 /= T3.sum(axis=3, keepdims=True) # normalize
np.nan_to_num(T3,copy=False)
# Removing and correcting nan entries
a,b,c,d = T3.shape
avg = 1 / (d)
for i in range(a):
    for j in range(b):
        for k in range(c):
            if abs(1 - sum(T3[i][j][k])) > 0.0001:
                T3[i][j][k] = np.full((d), avg)
assert((T3>=0).all())

In [27]:
# Given an amino acid sequence aaseq,
# determine the secondary structure sequence ssseq
def aa_quadgram_sample(aaseq):
    ssseq = np.array([])
    for i in range(0,len(aaseq)):
        if i == 0:
            ssseq = np.append(ssseq, np.random.choice(ssvals, p=T_aa[aaseq[i],:]) )
        elif i==1:
            ssseq = np.append(ssseq, np.random.choice(ssvals, p=T2[aaseq[i-1],aaseq[i],:]) )
        else:
            ssseq = np.append(ssseq, np.random.choice(ssvals, p=T3[aaseq[i-2],aaseq[i-1],aaseq[i],:]) )
    return ssseq

In [28]:
%%time
# Trained on random selection of 80% of the data set
errors = []
trials = 10
for i in range(len(XTe1)):
    ers = []
    for j in range(trials):
        ers.append(error(aa_quadgram_sample(XTe1[i]),YTe1[i]))
    errors.append(np.average(ers))
print("Amino Acid Quadgram Model (80:20)")
print("\nError rate:", np.average(errors))
print()

Amino Acid Quadgram Model (80:20)

Error rate: 0.28844038839728703

CPU times: user 1min 21s, sys: 391 ms, total: 1min 22s
Wall time: 1min 22s


#### Train/Test Set 2 (20:80)

In [29]:
# Count frequency of each transition for all proteins in training data
T3 = np.zeros( (len(aavals), len(aavals), len(aavals), len(ssvals)) )
for j in range(len(XTr2)):
    for i in range(0, len(XTr2[j])):
        if i == 0:
            T3[ 0 , 0, aavals.index(XTr2[j][i]), ssvals.index(YTr2[j][i]) ] += 1
        elif i==1:
            T3[ 0, aavals.index(XTr2[j][i-1]), aavals.index(XTr2[j][i]), ssvals.index(YTr2[j][i]) ] += 1
        else:
            T3[ aavals.index(XTr2[j][i-2]), aavals.index(XTr2[j][i-1]), aavals.index(XTr2[j][i]), ssvals.index(YTr2[j][i]) ] += 1
T3 /= T3.sum(axis=3, keepdims=True) # normalize
np.nan_to_num(T3,copy=False)
# Removing and correcting nan entries
a,b,c,d = T3.shape
avg = 1 / (d)
for i in range(a):
    for j in range(b):
        for k in range(c):
            if abs(1 - sum(T3[i][j][k])) > 0.0001:
                T3[i][j][k] = np.full((d), avg)
assert((T3>=0).all())

In [30]:
# Given an amino acid sequence aaseq,
# determine the secondary structure sequence ssseq
def aa_quadgram_sample(aaseq):
    ssseq = np.array([])
    for i in range(0,len(aaseq)):
        if i == 0:
            ssseq = np.append(ssseq, np.random.choice(ssvals, p=T_aa[aaseq[i],:]) )
        elif i==1:
            ssseq = np.append(ssseq, np.random.choice(ssvals, p=T2[aaseq[i-1],aaseq[i],:]) )
        else:
            ssseq = np.append(ssseq, np.random.choice(ssvals, p=T3[aaseq[i-2],aaseq[i-1],aaseq[i],:]) )
    return ssseq

In [31]:
%%time
# Trained on random selection of 80% of the data set
errors = []
trials = 10
for i in range(len(XTe2)):
    ers = []
    for j in range(trials):
        ers.append(error(aa_quadgram_sample(XTe2[i]),YTe2[i]))
    errors.append(np.average(ers))
print("Amino Acid Quadgram Model (20:80)")
print("\nError rate:", np.average(errors))
print()

Amino Acid Quadgram Model (20:80)

Error rate: 0.28597876694707375

CPU times: user 5min 17s, sys: 1.48 s, total: 5min 18s
Wall time: 5min 21s


# 6. Stochastic Random Model
In order to truly evaluate the quality of the previous models, we should compare them to an entirely random model

In [32]:
def random_sample(aaseq):
    ssseq = np.array([])
    for i in range(len(aaseq)):
        ssseq = np.append( ssseq, np.random.choice(ssvals) )
    return ssseq

In [33]:
%%time
errors = []
trials = 10
for i in range(len(X)):
    ers = []
    for j in range(trials):
        ers.append(error(random_sample(X[i]),Y[i]))
    errors.append(np.average(ers))
print("Stochastic Random Model")
print("Error rate:", np.average(errors))
print()
print()

Stochastic Random Model
Error rate: 0.12470606758441838


CPU times: user 3min 53s, sys: 1.17 s, total: 3min 54s
Wall time: 3min 55s
